In [1]:
# Plot project search history
import json
import util.plot
from pathlib import Path
from nn.hyperparameter.hyper_parameter import HyperParameter, ValueType, SearchType
from ast import literal_eval as make_tuple
from collections import namedtuple
import os

In [2]:
# plot functions
def plot_history(histories, title, path):
    # evaluate search history, do plots etc
    #title = "LSTM Loss (" + ", ".join([i.name for i in param_team]) + ")"
    #selected_params = '-'.join([i.name for i in param_team])
    # plot the histories
    lstm_history_plotter = util.plot.Plotter()
    lstm_history_plotter.plot_multi_loss_history(histories, highlight_best_count=3, title=title)
    lstm_history_plotter.show()
    lstm_history_plotter.save_figures(path, "history", "svg")

# https://jakevdp.github.io/PythonDataScienceHandbook/04.06-customizing-legends.html
# only supports two hyperparameters
def plot_scatter(histories, param_team, title, path):
    lstm_history_plotter = util.plot.Plotter()
    lstm_history_plotter.plot_multi_loss_scatter(histories, param_team, highlight_best_count=3, title=title)
    lstm_history_plotter.show()
    lstm_history_plotter.save_figures(path, "scatter", "svg")

In [3]:
def evaluate_search(search_path):
    print("\n============================================================================================")
    print("Evaluating '{}'".format(search_path))
    
    # load arguments.json and search_hist.json
    assert Path(search_path).exists(), "Project path '{}' not found.".format(search_path)
    if Path(search_path+"search_hist.json").exists() == False:
        return

    # load arguments
    args = None
    argument_file = search_path + "arguments.json"
    with open(argument_file) as f:
        args = json.load(f, object_hook=lambda d: namedtuple('args', d.keys())(*d.values()))

    # load hyperparameters from args
    param_team = []
    for param in args.lstm_parameter:
        search_type = param[1]
        if search_type == 0 or search_type == 1:
            hyper_param = HyperParameter(parameter_name=param[0], search_type=SearchType(param[1]), value_type=ValueType(param[2]), value_range=param[3], iterations=param[4])
        else:
            hyper_param = HyperParameter(parameter_name=param[0], search_type=SearchType(param[1]), value_type=ValueType(param[2]), values=param[3])
        param_team.append(hyper_param)

    # load history
    histories = None
    history_file = search_path + "search_hist.json"
    with open(history_file) as f:
        histories = json.load(f)
        histories = dict((make_tuple(key), value) for (key, value) in histories.items())

    # plot the history
    plot_history(histories, "Metrics", search_path)
    plot_scatter(histories, param_team, "Scatter", search_path)
    # best values
    best_entry = None
    best_val = float("inf")
    for parameter, hist in histories.items():
        for metric, value in hist.items():
            if "val_loss" == metric and value[-1] < best_val:
                best_val = value[-1]
                best_entry = (parameter,)
                for met, val in hist.items():
                    best_entry += (met, val[-1])

    print("Best Entry: {} -> {}".format(best_entry[0], str(best_entry[1:]) ))

In [4]:
# start the search
project_path = "../projects/adam_params/"
folder_name = "search"

if folder_name:
    evaluate_search(project_path+folder_name+"/")
else:
    # find all subfolders and evaluate search history
    filenames = os.listdir (project_path)
    for filename in filenames:
        joined_name = os.path.join(project_path, filename)
        if os.path.isdir(joined_name):
            evaluate_search(joined_name+"/")


Evaluating '../projects/adam_params/search/'


TypeError: 'NoneType' object is not iterable